In [ ]:
player_name = input("What is the name of the player you want to search for? ")
platform = input("What platform is the player on? ")
type = input("What is the type of the player? ")

In [ ]:
processed_player_name = player_name.replace(" ", "+")
processed_platform = platform.lower()
processed_type = type.lower()

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os

query = f"https://www.bcci.tv/search?platform={processed_platform}&type={processed_type}&term={processed_player_name}&content_type=all"

if not processed_player_name or not processed_platform or not processed_type:
    print("Invalid input")
    exit()
else:
    try:
        response = requests.get(query)
        response.raise_for_status()
    except requests.RequestException as e:
        print(e)

    soup = BeautifulSoup(response.text, 'html.parser')
    data = []

    player_data = soup.find('div', class_='lv pc video-section-append-video-here')
    if player_data:
        player_card = player_data.find_all(
            'div', class_='slick-card m-0 lv-bg hoverVideoPlayNow')
        for card in player_card:
            header_content = card.find('div', class_='bottom')
            title_holder = None
            title = ""
            date = ""
            views = ""
            if header_content.find('div', class_='text-detail br-b'):
                title_holder = header_content.find('div', class_='text-detail br-b')
                date_holder = header_content.find('div', class_='text-detail br-b')
                date =  date_holder.find('span').text
                views_holder = header_content.find(
                    'div', class_='tour-overlay-details')
                list = views_holder.find('ul')
                views = list.find_all('li')[0]
                views = views.find('span', class_="me-3").text
                views = views.replace("&nbsp;", "").replace(
                    "views", "").replace("\n", "").replace("\xa0", "").strip()
                if "k" in views:
                    views = views.replace("k", "")
                    views = float(views) * 1000
                elif "m" in views:
                    views = views.replace("m", "")
                    views = float(views) * 1000000
            elif header_content.find('div', class_='text-detail pb-0'):
                title_holder = header_content.find('div', class_='text-detail pb-0')
                date_holder = header_content.find(
                    'div', class_='tour-overlay-details')
                ul = date_holder.find('ul')
                date_ = ul.find_all('li')[0]
                date = date_.find('span').text
                views = "-"
            if title_holder:
                title = title_holder.find('p').text

            data.append({
                "title": title,
                "date": date,
                "views": views,
                "platform": processed_platform,
                "type": processed_type,
                "player_name": player_name,
                "sport": "Cricket"
            })
        if data:
            df = pd.DataFrame(data)
            header = ['title', 'date', 'views', 'platform', 'type', 'player_name']
            if os.path.isfile('Sports.csv') and os.path.getsize('Sports.csv') > 0:
                df.to_csv('Sports.csv', mode='a', header=False, index=False)
            else:
                df.to_csv('Sports.csv', mode='a', header=header, index=False)
    else:
        print("Player not found")

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

class ICC_Scrapper:
    def __init__(self):
        self.url = "https://www.icc-cricket.com/search?"
        self.player_name = None

    def get_player_data(self, player_name):
        processed_player_name = player_name.replace(" ", "%20")

        if not processed_player_name:
            return {"Error": "Invalid input"}

        self.player_name = processed_player_name

        url = self.url + "q=" + self.player_name

        response = self.get_data(processed_player_name, url)

        if response:
            return {"Response": response}
        else:
            return {"Error": "No data found"}

    def get_data(self, player_name, url):
        try:
            response = requests.get(url)
            response.raise_for_status()
        except requests.RequestException as e:
            return {"Error": str(e)}

        soup = BeautifulSoup(response.text, 'html.parser')
        data = []

        player_data = soup.find(
            'div', class_='my-4 grid grid-cols-1 lg:grid-cols-4 gap-6 lg:gap-x-6 lg:gap-y-10')
        if player_data:
            cards = player_data.find_all(
                'div', class_='h-[541px] relative rounded-lg lg:rounded-[14px] overflow-hidden')
            for card in cards:
                title = ""
                date = ""
                content = ""
                image_url = ""
                link = ""
                link_holder = card.find('a')
                link = link_holder['href']
                image_holder = link_holder.find_all(
                    'div')[0]
                image_holder = image_holder.find('picture')
                image_holder = image_holder.find('img')
                image_url = image_holder['src']
                divs = link_holder.find_all('div')
                if len(divs) > 1:
                    content_holder = divs[1]
                    content_holder = content_holder.find('div')
                    title_holder = content_holder.find_all('div')[1] if len(
                        content_holder.find_all('div')) > 1 else None
                    if title_holder:
                        title = title_holder.text
                    content_holder = content_holder.find(
                        'div', class_='text-sm font-bold text-white leading-[1.2] lg:text-lg lg:leading-[1.4] lg:-tracking-[0.72px]')
                    if content_holder:
                        content = content_holder.text
                    date_holder = content_holder.find('time')
                    date = date_holder.text if date_holder else ""
                else:
                    continue

                temp_player_name = player_name.replace("%20", " ")
                data.append({
                    "title": title,
                    "date": date,
                    "content": content,
                    "player_name": temp_player_name,
                    "image_url": image_url,
                    "link": link,
                    "sport": "Cricket"
                })

            if data:
                df = pd.DataFrame(data)
                header = ['title', 'date', 'content',
                          'player_name', 'image_url', 'link', 'sport']
                if os.path.isfile('ICC.csv') and os.path.getsize('ICC.csv') > 0:
                    df.to_csv('ICC.csv', mode='a', header=False, index=False)
                else:
                    df.to_csv('ICC.csv', mode='a', header=header, index=False)
                return data
            else:
                return {"Error": "No data found"}
        else:
            return {"Error": "No data found"}

In [ ]:
scrapper = ICC_Scrapper()

# player_name = "MS Dhoni"
# scrapper.get_player_data(player_name)

player_name = "Sachin Tendulkar"
scrapper.get_player_data(player_name)

# player_name = "Sourav Ganguly"
# scrapper.get_player_data(player_name)

# player_name = "Ravindra Jadeja"
# scrapper.get_player_data(player_name)

# player_name = "Jasprit Bumrah"
# scrapper.get_player_data(player_name)

# player_name = "Mithali Raj"
# scrapper.get_player_data(player_name)

# player_name = "Smriti Mandhana"
# scrapper.get_player_data(player_name)

# player_name = "Jhulan Goswami"
# scrapper.get_player_data(player_name)

# player_name = "Harmanpreet Kaur"
# scrapper.get_player_data(player_name)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

class FIFA_Scrapper:
    def __init__(self):
        self.url = "https://www.fifa.com/en/search?"
        self.player_name = None

    def get_player_data(self, player_name):
        processed_player_name = player_name.replace(" ", "%20")

        if not processed_player_name:
            return {"Error": "Invalid input"}

        self.player_name = processed_player_name

        url = self.url + "q=" + self.player_name

        response = self.get_data(processed_player_name, url)

        if response:
            return {"Response": response}
        else:
            return {"Error": "No data found"}

    def get_data(self, player_name, url):
        try:
            response = requests.get(url)
            response.raise_for_status()
        except requests.RequestException as e:
            return {"Error": str(e)}

        soup = BeautifulSoup(response.text, 'html.parser')
        data = []

        player_data = soup.find(
            'div', class_='search-results-page_searchResults__pKRI-')
        if player_data:
            title = ""
            date = ""
            content = ""
            image_url = ""
            link = ""
            player_name = ""
            sport = "Football"
            cards = player_data.find_all('a')
            for card in cards:
                link = card['href']
                date_holder = card.find(
                    'div', class_='search-result-card_details__+1JIM')
                date = date_holder.find('span')[1].text
                image_holder = card.find(
                    'div', class_='search-result-card_imageContainer__NgxbS')
                image_holder = image_holder.find(
                    'div', class_='image_imgContainer__nDjya')
                image_holder = image_holder.find('img')
                image_url = image_holder['src']
                content_holder = card.find(
                    'div', class_='search-result-card_textContainer__82BrL')
                title_holder = content_holder.find(
                    'div', class_='search-result-card_title__CTmi4')
                title = title_holder.text
                content_holder = content_holder.find(
                    'div', class_='search-result-card_description__KiKIZ search-result-card_smallCardDescription__InIop')
                content = content_holder.text
                temp_player_name = player_name.replace("%20", " ")
                data.append({
                    "title": title,
                    "date": date,
                    "content": content,
                    "player_name": temp_player_name,
                    "image_url": image_url,
                    "link": link,
                    "sport": sport
                })

            if data:
                df = pd.DataFrame(data)
                header = ['title', 'date', 'content',
                          'player_name', 'image_url', 'link', 'sport']
                if os.path.isfile('FIFA.csv') and os.path.getsize('FIFA.csv') > 0:
                    df.to_csv('FIFA.csv', mode='a', header=False, index=False)
                else:
                    df.to_csv('FIFA.csv', mode='a', header=header, index=False)
                return data
            else:
                return {"Error": "No data found"}
        else:
            return {"Error": "No data found"}

In [ ]:
scrapper = FIFA_Scrapper()

player_name = "Lionel Messi"
scrapper.get_player_data(player_name)

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os


class ICAR_Scrapper:
    def __init__ (self):
        self.url = "https://www.icar.org.in/search/node?keys="
        self.query = None

    def get_query_data(self, query):
        processed_query = query.replace(" ", "%20")

        if not processed_query:
            return {"Error": "Invalid input"}

        self.query = processed_query

        url = self.url + processed_query

        response = self.get_data(processed_query, url)

        if response:
            return {"Response": response}
        else:
            return {"Error": "No data found"}
        
    def get_data(self, query, url):
        try:
            response = requests.get(url, verify=False)
            response.raise_for_status()
        except requests.RequestException as e:
            return {"Error": str(e)}
        
        soup = BeautifulSoup(response.text, 'html.parser')
        data = []

        query_data = soup.find(
            'ol', class_='search-results node_search-results')
        if query_data:
            title = ""
            date = ""
            content = ""
            link = ""
            place = ""

            li_tags = query_data.find_all('li')
            if li_tags:
                for li in li_tags:
                    title_holder = li.find('h3')
                    title_holder = title_holder.find('a')
                    link = title_holder['href']
                    title = title_holder.text

                    home_page = requests.get(link, verify=False)
                    home_page.raise_for_status()
                    home_soup = BeautifulSoup(home_page.text, 'html.parser')

                    content_holder = home_soup.find(
                        'div', class_='node__content clearfix')
                    if content_holder:
                        content_holder = content_holder.find(
                            'div', class_='clearfix text-formatted field field--name-body field--type-text-with-summary field--label-hidden field__item')
                    
                    if content_holder:
                        paragraphs = content_holder.find_all('p')
                        if paragraphs[0].find('em'):
                            date_place = paragraphs[0].find('em')
                            date_place = date_place.text   
                            date_place = date_place.split(',')
                            if len(date_place) == 2:
                                if any(char.isdigit() for char in date_place[0]):
                                    date = date_place[0]
                                    place = date_place[1]
                                else:
                                    date = date_place[1]
                                    place = date_place[0]
                            elif len(date_place) == 3:
                                date = date_place[0] + ', ' + date_place[1]
                                place = date_place[2]
                            content = ""
                            for i in range(1, len(paragraphs)):
                                content += paragraphs[i].text + " "
                        else:
                            date = "-"
                            place = "-"
                            content = ""
                            for i in range(0, len(paragraphs)):
                                content += paragraphs[i].text + " "

                    data.append({
                        "title": title,
                        "date": date,
                        "content": content,
                        "link": link,
                        "place": place,
                        "query": query,
                        "category": "Agriculture"
                    })

            if data:
                df = pd.DataFrame(data)
                header = ['title', 'date', 'content', 'link', 'place', 'query', 'category']
                if os.path.isfile('ICAR.csv') and os.path.getsize('ICAR.csv') > 0:
                    df.to_csv('ICAR.csv', mode='a', header=False, index=False)
                else:
                    df.to_csv('ICAR.csv', mode='a', header=header, index=False)
                return data
            else:
                return {"Error": "No data found"}
        else:
            return {"Error": "No data found"}
        

In [4]:
scraper = ICAR_Scrapper()

query = "Grapes"
scraper.get_query_data(query)

c:\Users\Aditya\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.icar.org.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\Aditya\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.icar.org.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\Aditya\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.icar.org.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.

{'Response': [{'title': 'Webinar on “Export of Grapes in 2021: Instructions for uses of authorized agrochemicals as per Annexure - 5 of Residue Monitoring Plan”',
   'date': '20th\xa0August,  2020',
   'content': 'The ICAR-National Research Centre on Grapes, Pune, Maharashtra organized a Webinar on\xa0“Export of Grapes in 2021: Instructions for uses of authorized agrochemicals as per Annexure - 5 of Residue Monitoring Plan”\xa0today. In his address, Dr. Anand Kumar Singh, Deputy Director General (Horticultural Science), ICAR emphasized on the importance of following the good agricultural practices for pesticide applications in grape cultivation. Dr. Singh also urged the participants for providing the technologies and recommendations for effective management of pesticide residues in grapes. Shri Devendra Prasad, DGM, APEDA along with the members of Maharashtra State Grape Growers’ Association (MRDBS) also virtually participated in the Webinar. Earlier, in his welcome address, Dr. R.G. S

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import os

class FAO_Scrapper:
    def __init__ (self):
        self.url = "https://www.fao.org/home/search/en/?"
        self.query = None

    def get_query_data(self, query):
        processed_query = query.replace(" ", "+")

        if not processed_query:
            return {"Error": "Invalid input"}

        self.query = processed_query

        url = self.url + "q=" + processed_query
        response = self.get_data(processed_query, url)
        
        if response:
            return {"Response": response}
        else:
            return {"Error": "No data found"}
        
    def get_data(self, query, url):
        try:
            driver = webdriver.Edge()
            driver.get(url)
        except requests.RequestException as e:
            return {"Error": str(e)}

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        data = []

        query_data = soup.find('div', class_='gsc-expansionArea')
        if query_data:
            cards = query_data.find_all('div', class_='gsc-webResult gsc-result')
            for card in cards:
                title = ""
                content = ""
                description = ""
                link = ""
                category = "Agriculture"

                card = card.find('div', class_='gs-webResult gs-result')
                header = card.find('div', class_='gsc-thumbnail-inside')
                title_holder = header.find('div', class_='gs-title')
                a_tag = title_holder.find('a')
                link = a_tag['href']
                title = a_tag.text

                home_driver = webdriver.Edge()
                home_driver.get(link)
                home_soup = BeautifulSoup(home_driver.page_source, 'html.parser')

                content_holder = home_soup.find('section', id='content')
                if content_holder:
                    content_holder = content_holder.find(
                        'div', class_='main-internal')
                    if content_holder:
                        desc_holder = content_holder.find(
                            'div', class_='csc-default')
                        if desc_holder:
                            desc_holder = desc_holder.find('p', class_='bodytext')
                            if desc_holder:
                                description = desc_holder.text
                            else :
                                description = "-"
                        
                        content_holder = content_holder.find_all(
                            'div', class_='rgaccord1-nest')
                        if content_holder:
                            content = ""
                            for div in content_holder:
                                section_title_holder = div.find('h3')
                                section_title = section_title_holder.text

                                text_holder = div.find('div')
                                text_holder = text_holder.find('div')
                                text = ''
                                
                                if text_holder:
                                    text = text_holder.text
                                

                                content += section_title + "\n" + text + "\n\n"

                data.append({
                    "title": title,
                    "content": content,
                    "description": description,
                    "link": link,
                    "query": query,
                    "category": category
                })
            home_driver.quit()
            driver.quit()
            if data:
                df = pd.DataFrame(data)
                header = ['title', 'content', 'description', 'link', 'query', 'category']
                if os.path.isfile('FAO.csv') and os.path.getsize('FAO.csv') > 0:
                    df.to_csv('FAO.csv', mode='a', header=False, index=False)
                else:
                    df.to_csv('FAO.csv', mode='a', header=header, index=False)
                return data
            else:
                return {"Error": "No data found"}
            
        else:
            return {"Error": "No data found"}
        


In [4]:
scrapper = FAO_Scrapper()

query = "Grapes"
scrapper.get_query_data(query)

{'Response': [{'title': 'Grape - Food and Agriculture Organization of the United Nations',
   'content': 'Crop Description and Climate\nCrop Description and ClimateGrape is believed to originate from the Caspian and Caucasian regions. Most cultivated vines belong to the European type (Vitis vinefera), the American bunch type (V. labrusca\xa0and its derivatives) or Muscadine type (V. rotundifolia). The total production of grapes is about 61.95 million tons from 7.3 million ha. (FAOSTAT, 2001)\n\nGrape is grown between about 50°N and S, with suitable areas being small at these limits. The crop needs a long, warm to hot, dry summer and a cool winter. The subtropics with winter rain are most suited. Rain or cold and cloudy weather during flowering may adversely affect fruit setting whereas rain during ripening may lead to fruit rot. Where raisins are produced by sun-drying between the vine rows, at least one warm, sunny month without rain following harvest is essential.\n\nIn climates with

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os


class Investopedia_Scrapper:
    def __init__(self):
        self.url = "https://www.investopedia.com/search?"
        self.query = None

    def get_query_data(self, query):
        processed_query = query.replace(" ", "+")

        if not processed_query:
            return {"Error": "Invalid input"}

        self.query = processed_query

        url = self.url + "q=" + processed_query

        response = self.get_data(processed_query, url)

        if response:
            return {"Response": response}
        else:
            return {"Error": "No data found"}

    def get_data(self, query, url):
        try:
            response = requests.get(url)
            response.raise_for_status()
        except requests.RequestException as e:
            return {"Error": str(e)}

        soup = BeautifulSoup(response.text, 'html.parser')
        data = []
        
        query_data = soup.find('div', class_='comp search-results__results mntl-block')
        if query_data:
            cards = query_data.find_all(
                'div', class_='comp search-results__list mntl-block')
            for card in cards:
                title = ""
                desc = ""
                date = ""
                author = ""
                author_link = ""
                reviewer = ""
                reviewer_link = ""
                content = ""
                link = ""
                keypoints = ""
                category = "Finance"

                link_holder = card.find('a')
                link = link_holder['href']
                title_holder = link_holder.find('h3')
                title = title_holder.text
                desc_holder = card.find(
                    'div', class_='comp search-results__description mntl-text-block')
                desc = desc_holder.text

                home_page = requests.get(link)
                home_page.raise_for_status()
                home_soup = BeautifulSoup(home_page.text, 'html.parser')

                content_holder = home_soup.find(
                    'div', class_='loc article-content')
                if content_holder:
                    content_holder = content_holder.find(
                        'div', class_='comp article-body mntl-block')
                    if content_holder:
                        content_holder = content_holder.find(
                            'div', class_='comp mntl-sc-page mntl-block article-body-content')
                        if content_holder:
                            paragraphs = content_holder.find_all('p')
                            content = ""
                            for p in paragraphs:
                                content += p.text + " "
                            keypoints = content_holder.find('div', class_='comp mntl-sc-key-points mntl-block')
                            if keypoints:
                                keypoints = keypoints.find(
                                    'div', class_='comp mntl-sc-block mntl-sc-block-callout mntl-block theme-whatyouneedtoknow')
                                if keypoints:
                                    keypoints = keypoints.find(
                                        'div', class_='comp mntl-sc-block-callout-body mntl-text-block')
                                    if keypoints:
                                        keypoints = keypoints.find('ul')
                                        if keypoints:
                                            keypoints = keypoints.find_all('li')
                                            keys = ""
                                            for key in keypoints:
                                                keys += key.text + " "
                                            keypoints = keys

                header_holder = home_soup.find(
                    'header', class_='loc article-pre-content')
                if header_holder:
                    header_holder = header_holder.find(
                        'header', class_='comp article-header mntl-block right-rail__offset js-toc-appear')
                    if header_holder:
                        meta_holder = header_holder.find('div', class_='comp article-meta mntl-block')
                        if meta_holder:
                            meta_holder = meta_holder.find(
                                'div', class_='comp finance-bylines mntl-bylines')
                            if meta_holder:
                                author_and_date = meta_holder.find('div', class_='comp mntl-bylines__group mntl-block mntl-bylines__group--author')
                                if author_and_date:
                                    author_holder = author_and_date.find('div', class_='comp mntl-bylines__item mntl-attribution__item mntl-attribution__item--has-date')
                                    if author_holder:
                                        author_holder = author_holder.find(
                                            'div', class_='mntl-dynamic-tooltip--trigger')
                                        if author_holder:
                                            author_holder = author_holder.find('a')
                                            author = author_holder.text
                                            author_link = author_holder['href']
                                    date_holder = author_and_date.find(
                                        'div', class_='mntl-attribution__item-date')
                                    if date_holder:
                                        date = date_holder.text
                                reviewer_holder = meta_holder.find(
                                    'div', class_='comp mntl-bylines__group mntl-block mntl-bylines__group--finance_reviewer')
                                if reviewer_holder:
                                    reviewer_holder = reviewer_holder.find(
                                        'div', class_='comp mntl-bylines__item mntl-attribution__item')
                                    if reviewer_holder:
                                        reviewer_holder = reviewer_holder.find(
                                            'div', class_='mntl-dynamic-tooltip--trigger')
                                        if reviewer_holder:
                                            reviewer_holder = reviewer_holder.find('a')
                                            reviewer = reviewer_holder.text
                                            reviewer_link = reviewer_holder['href']

                data.append({
                    "title": title,
                    "description": desc,
                    "date": date,
                    "author": author,
                    "author_link": author_link,
                    "reviewer": reviewer,
                    "reviewer_link": reviewer_link,
                    "content": content,
                    "link": link,
                    "keypoints": keypoints,
                    "query": query,
                    "category": category
                })


            if data:
                df = pd.DataFrame(data)
                header = [
                    'title', 'description', 'date', 'author', 'author_link', 'reviewer', 'reviewer_link', 'content', 'link', 'keypoints', 'query', 'category']
                if os.path.isfile('Investopedia.csv') and os.path.getsize('Investopedia.csv') > 0:
                    df.to_csv('Investopedia.csv', mode='a', header=False, index=False)
                else:
                    df.to_csv('Investopedia.csv', mode='a', header=header, index=False)
                return data
            else:
                return {"Error": "No data found"}
        else:
            return {"Error": "No data found"}

In [2]:
scrapper = Investopedia_Scrapper()

query = "Mukesh Ambani"
scrapper.get_query_data(query)

{'Response': {'Error': '406 Client Error: Not Acceptable for url: https://www.investopedia.com/search?q=Mukesh+Ambani'}}